### Sri Lanka Road Optimization
This notebook is part of a Sri Lanka tourism project that measures the distance from airports to district capitals to toursim destinations. 

#### Workflow
- part 1: OD Airport to major cities
- part 2: OD Major cities to one another
- part 3: OD Major cities to tourism destinations
- part 4*: OD using mapbox traffic api OR mapbox roads data w/ traffic (2-3x for different times of day)
- part 5*: Identify optimal edges for road rehabilitation (traffic travel time is divergent)
- part 6*: OD - simulate road improvements with adjusted cost
- part 7*: identify possible edges for improvement (highest value for money). 
#### *To-do


#### to run this script you must already have the G_time.pickle (from data folder)


In [1]:
# import modules and dependencies

import geopandas as gpd
import pandas as pd
import os, sys, time
# add to your system path the location of the LoadOSM.py and GOSTnet.py scripts
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), r"C:\Users\wb557957\Desktop\GOSTnets-master"))

import GOSTnets as gn
from scipy import *
import GOSTnets.network_clean as gnClean
import importlib
import networkx as nx
import osmnx as ox
from shapely.ops import unary_union
from shapely.wkt import loads
from shapely.geometry import LineString, MultiLineString, Point

In [2]:
# have jupyter reload all modules when you run the code
%load_ext autoreload
%autoreload 2

In [3]:
pth = r"C:\Users\wb557957\Desktop\GOSTnets-master\LKA"

In [4]:
# speed dictionary for reference
# all speeds are the mean value of roads found in OSM.
'''speed_dict = {
                'primary': 60,
                'primary_link': 50,
                'secondary': 50,
                'secondary_link': 45,
                'tertiary':40,
                'tertiary_link': 40,
                'residential': 30,
                'trunk': 60,
                'trunk_link': 50,
                'unclassified': 25,
                'track': 25,
                'service': 20
                }
                '''

In [4]:
G_time = nx.read_gpickle(os.path.join(pth, r'G_time.pickle'))

### part 1: OD airport to major cities

In [10]:
# read in single origin (Ratmalana Airport, Colombo, Sri Lanka) 
airport = gpd.read_file(os.path.join(pth, 'lka_data', 'LKA_airport.shp'))

airport

,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,geometry
0,2.700505e+09,None,gate,None,None,None,None,None,None,None,None,POINT (79.88462 7.17607)


In [11]:
# add X Y to airport object as columns
airport['x']=airport.geometry.x
airport['y']=airport.geometry.y

airport

,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,geometry,x,y
0,2.700505e+09,None,gate,None,None,None,None,None,None,None,None,POINT (79.88462 7.17607),79.884621,7.176073


In [12]:
# snap airport to G_time based on nearest node.
airport = gn.pandana_snap_c(G_time, 
                            airport, 
                            source_crs='epsg:4326',
                            target_crs='epsg:32644',
                            add_dist_to_node_col=True)

airport

C:\WBG\Anaconda\envs\test_gostNets\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


,osm_id,emergencyh,aeroway,operatorty,source,capacitype,addrfull,emergency,name,addrcity,building,geometry,x,y,NN,NN_dist
0,2.700505e+09,None,gate,None,None,None,None,None,None,None,None,POINT (79.88462 7.17607),79.884621,7.176073,18713,191.443706


In [13]:
# create list of airport that can be read into the OD calculation. 
# is there a way to create a list of airports with NN and other properties (osm_id)?
airport_ls = list(set(airport.NN))

# prints NN for the airport
airport_ls

[18713]

In [17]:
# input major cities as destinations
city = gpd.read_file(os.path.join(pth, 'lka_data', 'LKA_distCapitals1.shp'))

# snap the tourism to the nearest network node:
city = gn.pandana_snap_c(G_time, 
                               city,
                               source_crs='epsg:4326',
                               target_crs='epsg:32644',
                               add_dist_to_node_col = True)

city

C:\WBG\Anaconda\envs\test_gostNets\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


,osm_id,code,fclass,population,name,country,geometry,NN,NN_dist
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382),45673,2.818824
1,49374844,1001,city,17900,Kegalle,LKA,POINT (80.34541 7.25320),108460,27.361894
2,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500),117093,14.243141
3,247220841,1001,city,120000,Hambantota,LKA,POINT (81.12426 6.12491),48165,52.511138
4,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213),64874,82.604133
5,258523517,1001,city,15000,Polonnaruwa,LKA,POINT (81.00034 7.93954),68568,4.854156
6,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019),285275,7.214712
7,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643),215717,7.611852
8,566574942,1001,city,63200,Anuradhapura,LKA,POINT (80.41061 8.33498),230721,18.559270
9,633759246,1001,city,100000,Mullaitivu,LKA,POINT (80.81453 9.26985),136052,10.682812


In [18]:
# add X Y to city object as columns
city['x']=city.geometry.x
city['y']=city.geometry.y
city

,osm_id,code,fclass,population,name,country,geometry,NN,NN_dist,x,y
0,44240758,1001,city,27500,Nuwara Eliya,LKA,POINT (80.76707 6.97382),45673,2.818824,80.767067,6.973816
1,49374844,1001,city,17900,Kegalle,LKA,POINT (80.34541 7.25320),108460,27.361894,80.345413,7.253201
2,50794342,1005,national_capital,753000,Colombo,LKA,POINT (79.85385 6.93500),117093,14.243141,79.853846,6.934997
3,247220841,1001,city,120000,Hambantota,LKA,POINT (81.12426 6.12491),48165,52.511138,81.124256,6.124913
4,258049423,1001,city,40800,Matale,LKA,POINT (80.62342 7.47213),64874,82.604133,80.623417,7.472125
5,258523517,1001,city,15000,Polonnaruwa,LKA,POINT (81.00034 7.93954),68568,4.854156,81.000339,7.939536
6,330173100,1001,city,45400,Puttalam,LKA,POINT (79.82866 8.03019),285275,7.214712,79.828658,8.030186
7,335253618,1001,city,99100,Trincomalee,LKA,POINT (81.23450 8.57643),215717,7.611852,81.234495,8.576425
8,566574942,1001,city,63200,Anuradhapura,LKA,POINT (80.41061 8.33498),230721,18.559270,80.410610,8.334985
9,633759246,1001,city,100000,Mullaitivu,LKA,POINT (80.81453 9.26985),136052,10.682812,80.814535,9.269853


In [21]:
# create list of destinations using nearest network node. 
city_ls = list(set(list(city.NN)))

# try creating a list with multiple field names in it: 
# city_ls = list(set(list([city.NN, city.name, city.code])))
# didn't work :/

city_ls

[59905,
 237826,
 164101,
 262918,
 19847,
 281879,
 210210,
 48165,
 215717,
 106667,
 108460,
 237504,
 230721,
 107723,
 136272,
 270164,
 257368,
 68568,
 285275,
 315998,
 117093,
 45673,
 64874,
 136052,
 151421,
 328574]

In [59]:
# output is a numpy array of OD time values
# 'AC' stands for Airport-to-City
OD_AC = gn.calculate_OD(G_time,
                     airport_ls, 
                     city_ls, 
                     fail_value = 9999999)

OD_AC

array([[20366.67769355,  1593.11754283, 14299.71792606, 11640.41482875,
        13324.80816559,  9001.44804346,  6687.43835971, 14196.21854542,
        14808.04388448,   830.47245897,  4382.98547231, 15771.44757046,
        11074.4590583 ,  4958.21109398, 13813.31358828,  4552.54325055,
        18866.69673784, 12244.82153827,  6549.53321352, 17849.11421183,
         2033.42807732, 10081.97267594,  8177.71867361, 19347.57974299,
        17653.35809136,  6625.13045935]])

In [60]:
# get shape of OD matrix to confirm count of O&D
OD_AC.shape

# 26 observations

(1, 26)

In [61]:
# convert the OD Matrix numpy array into a pandas dataframe using minutes as the measure
# output: minutes from airport to each destination NN id (n=26)
OD_AC = OD_AC / 60

OD_dfAC = pd.DataFrame(OD_AC, columns = city_ls, index = airport_ls)

OD_dfAC

,59905,237826,164101,262918,19847,281879,210210,48165,215717,106667,...,257368,68568,285275,315998,117093,45673,64874,136052,151421,328574
18713,339.444628,26.551959,238.328632,194.006914,222.080136,150.024134,111.457306,236.603642,246.800731,13.841208,...,314.444946,204.080359,109.158887,297.485237,33.890468,168.032878,136.295311,322.459662,294.222635,110.418841


In [62]:
# transpose the df and save as an object 
OD_dfAC = OD_dfAC.transpose()

In [63]:
OD_dfAC

,18713
59905,339.444628
237826,26.551959
164101,238.328632
262918,194.006914
19847,222.080136
281879,150.024134
210210,111.457306
48165,236.603642
215717,246.800731
106667,13.841208


### part 2: OD cities to other cities

In [64]:
# in part 1, we've already imported, snapped and created a list from our district capitals (cities) dataset.
# in this process, we will call the cities into the OD calculation as both origin and destiation
# 'CC' stands for City-to-City
OD_CC = gn.calculate_OD(G_time,
                     city_ls, 
                     city_ls, 
                     fail_value = 9999999)

OD_CC

array([[    0.        , 21308.15306012,  8734.97576252, 31325.76708999,
        25014.40599086, 28686.80030471, 25802.9259326 , 32833.81819777,
        14600.66086806, 19718.43198335, 20334.90141759, 27018.05444996,
        11998.61648279, 18399.18378228,  7148.72131056, 24237.89551179,
         4167.66525968, 17311.74047964, 13817.6775029 , 25307.72261215,
        21718.78033857, 23911.53869669, 17994.45752545,  6985.95296322,
        22260.63321215, 19553.32149321],
       [21308.15306012,     0.        , 13810.05937274, 11379.59600967,
        12083.94543978,  8740.62922438,  5313.04883986, 12821.82902557,
        14024.77202508,  1771.93533196,  3176.53124382, 14397.05805061,
        10584.80050497,  4151.90073704, 14754.78895485,  4291.72443147,
        18377.03818451, 11461.54967887,  7491.00858009, 16642.65998334,
         1806.67344447,  8875.51844745,  6971.26444511, 18857.92118967,
        16446.90386287,  5418.67623086],
       [ 8734.97576252, 13810.05937274,     0.        

In [65]:
OD_CC.shape

(26, 26)

In [66]:
OD_CC = OD_CC / 60

OD_dfCC = pd.DataFrame(OD_CC, columns = city_ls, index = city_ls)

OD_dfCC

,59905,237826,164101,262918,19847,281879,210210,48165,215717,106667,...,257368,68568,285275,315998,117093,45673,64874,136052,151421,328574
59905,0.000000,355.135884,145.582929,522.096118,416.906767,478.113338,430.048766,547.230303,243.344348,328.640533,...,69.461088,288.529008,230.294625,421.795377,361.979672,398.525645,299.907625,116.432549,371.010554,325.888692
237826,355.135884,0.000000,230.167656,189.659933,201.399091,145.677154,88.550814,213.697150,233.746200,29.532256,...,306.283970,191.025828,124.850143,277.377666,30.111224,147.925307,116.187741,314.298686,274.115064,90.311271
164101,145.582929,230.167656,0.000000,413.250170,271.329365,369.267390,284.471364,401.652902,97.761598,228.098207,...,76.121842,142.951607,130.678432,276.217976,253.701460,252.948244,154.330224,84.136558,225.427804,180.311290
262918,522.096118,189.659933,413.250170,0.000000,202.976748,43.998441,147.933059,83.915903,416.828714,196.481206,...,489.366483,349.639336,291.810377,281.211092,162.175504,231.250289,299.270254,497.381200,339.286917,273.393784
19847,416.906767,201.399091,271.329365,202.976748,0.000000,246.095506,125.490261,130.599912,254.998333,225.754325,...,347.445679,146.938962,242.145874,134.965767,210.878422,56.241480,138.350506,355.460396,159.300438,114.760639
281879,478.113338,145.677154,369.267390,43.998441,246.095506,0.000000,150.393690,127.914344,372.845934,152.498427,...,445.383704,330.125562,247.827597,324.669829,118.192724,257.372815,255.287475,453.398420,382.745654,229.411004
210210,430.048766,88.550814,284.471364,147.933059,125.490261,150.393690,0.000000,125.154356,287.944311,115.233215,...,360.587678,245.223938,209.934686,244.596627,88.318921,139.372664,157.467591,368.602395,284.514325,131.591121
48165,547.230303,213.697150,401.652902,83.915903,130.599912,127.914344,125.154356,0.000000,370.345670,240.379551,...,477.769216,277.262499,335.081022,204.139177,213.465257,166.852201,265.175338,485.783932,262.215002,239.298867
215717,243.344348,233.746200,97.761598,416.828714,254.998333,372.845934,287.944311,370.345670,0.000000,240.393956,...,173.883260,108.082322,183.541257,203.382677,257.280005,256.421190,157.803171,146.299286,136.511782,183.784237
106667,328.640533,29.532256,228.098207,196.481206,225.754325,152.498427,115.233215,240.379551,240.393956,0.000000,...,304.214521,197.673584,98.354792,299.260086,36.364761,169.807727,131.488945,312.229238,292.955557,112.193691


In [67]:
OD_dfCC = OD_dfCC.transpose()

In [68]:
OD_dfCC 

            59905       237826      164101      262918      19847   \
59905     0.000000  355.135884  145.582929  522.096118  416.906767   
237826  355.135884    0.000000  230.167656  189.659933  201.399091   
164101  145.582929  230.167656    0.000000  413.250170  271.329365   
262918  522.096118  189.659933  413.250170    0.000000  202.976748   
19847   416.906767  201.399091  271.329365  202.976748    0.000000   
281879  478.113338  145.677154  369.267390   43.998441  246.095506   
210210  430.048766   88.550814  284.471364  147.933059  125.490261   
48165   547.230303  213.697150  401.652902   83.915903  130.599912   
215717  243.344348  233.746200   97.761598  416.828714  254.998333   
106667  328.640533   29.532256  228.098207  196.481206  225.754325   
108460  338.915024   52.942187  193.337622  236.024701  150.931480   
237504  450.300907  239.950968  304.723506  188.022639   58.475663   
230721  199.976941  176.413342   54.399540  359.495855  231.533813   
107723  306.653063  

In [80]:
# to run a check on your outputs, find locations for city with NN id: 59905
index=city_ls

pd.DataFrame.sort_values(OD_dfCC, by=index, ascending=True)

,59905,237826,164101,262918,19847,281879,210210,48165,215717,106667,...,257368,68568,285275,315998,117093,45673,64874,136052,151421,328574
59905,0.000000,355.135884,145.582929,522.096118,416.906767,478.113338,430.048766,547.230303,243.344348,328.640533,...,69.461088,288.529008,230.294625,421.795377,361.979672,398.525645,299.907625,116.432549,371.010554,325.888692
257368,69.461088,306.283970,76.121842,489.366483,347.445679,445.383704,360.587678,477.769216,173.883260,304.214521,...,0.000000,219.067920,206.794746,352.334289,329.817774,329.064557,230.446538,58.642656,301.549466,256.427604
136052,116.432549,314.298686,84.136558,497.381200,355.460396,453.398420,368.602395,485.783932,146.299286,312.229238,...,58.642656,227.082637,214.809463,344.359209,337.832491,337.079274,238.461255,0.000000,277.488314,264.442321
136272,119.145355,245.913149,77.656585,412.873383,331.049291,368.890603,330.974782,456.121118,175.418003,219.417798,...,117.440068,202.671532,121.071890,335.937901,252.756937,312.668169,214.050150,143.567974,297.953506,240.031216
164101,145.582929,230.167656,0.000000,413.250170,271.329365,369.267390,284.471364,401.652902,97.761598,228.098207,...,76.121842,142.951607,130.678432,276.217976,253.701460,252.948244,154.330224,84.136558,225.427804,180.311290
230721,199.976941,176.413342,54.399540,359.495855,231.533813,315.513076,233.027318,358.173655,107.262365,174.343893,...,130.515854,103.156054,76.924118,236.422423,199.947146,213.152691,114.534672,138.530570,198.438028,140.515738
285275,230.294625,124.850143,130.678432,291.810377,242.145874,247.827597,209.934686,335.081022,183.541257,98.354792,...,206.794746,176.928026,0.000000,310.194394,131.693931,196.771504,141.789416,214.809463,272.209999,128.636727
215717,243.344348,233.746200,97.761598,416.828714,254.998333,372.845934,287.944311,370.345670,0.000000,240.393956,...,173.883260,108.082322,183.541257,203.382677,257.280005,256.421190,157.803171,146.299286,136.511782,183.784237
68568,288.529008,191.025828,142.951607,349.639336,146.938962,330.125562,245.223938,277.262499,108.082322,197.673584,...,219.067920,0.000000,176.928026,133.289320,214.559632,185.146547,115.082798,227.082637,95.304925,141.063864
64874,299.907625,116.187741,154.330224,299.270254,138.350506,255.287475,157.467591,265.175338,157.803171,131.488945,...,230.446538,115.082798,141.789416,199.702427,139.721545,98.618020,0.000000,238.461255,196.439825,25.981066


### part 3: OD cities to destinations
in part 3, we calculate travel time for all cities to all destinations

there were are a total of 414 geocoded sites, but only 336 had unique nearest nodes.


In [81]:
# import destinations shapefile
tourism = gpd.read_file(os.path.join(pth, 'lka_data', 'tourismDestinations_geocoded_030420.shp'))

# add X Y columns to object
tourism['x']=tourism.geometry.x
tourism['y']=tourism.geometry.y

#snap to G_time
tourism = gn.pandana_snap_c(G_time, 
                            tourism, 
                            source_crs='epsg:4326',
                            target_crs='epsg:32644',
                            add_dist_to_node_col=True)

C:\WBG\Anaconda\envs\test_gostNets\lib\site-packages\pyproj\crs.py:77: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method.
  return _prepare_from_string(" ".join(pjargs))


In [82]:
tourism

,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,...,USER_langC,USER_adm1c,USER_adm2c,USER_adm3c,USER_adm4c,geometry,x,y,NN,NN_dist
0,World,M,80.01,A,Embekka Katharagama Dewalaya,"Embekka Katharagama Dewalaya, Embekka, LKA",Embekka Katharagama Dewalaya,POI,Temple,Embekka Katharagama Dewalaya,...,ENG,LK2,LK21,None,None,POINT (80.56806 7.21811),80.568060,7.218110,44921,79.587877
1,None,M,100.00,PP,"x=80.742936, y=7.400645",None,None,None,None,None,...,ENG,LK2,LK21,None,None,POINT (80.74294 7.40064),80.742936,7.400645,73021,130.113072
2,World,M,87.33,A,Royal Palace Park-Kandy,"Royal Palace Park-Kandy, Keerthie Sri Rajasing...",Royal Palace Park-Kandy,POI,Park,Royal Palace Park-Kandy,...,ENG,LK2,LK21,None,None,POINT (80.63751 7.28947),80.637510,7.289470,191838,54.272482
3,World,M,75.48,M,Kandy,"Kandy, LKA",Kandy,Locality,City,Kandy,...,ENG,LK2,LK21,None,None,POINT (80.63866 7.29497),80.638660,7.294970,133007,45.430757
4,World,M,65.07,A,Bauddha Bala Mandalaya,"Bauddha Bala Mandalaya, Akkara Road, Ranawana ...",Bauddha Bala Mandalaya,POI,Temple,Bauddha Bala Mandalaya,...,ENG,LK2,LK21,None,None,POINT (80.62270 7.32835),80.622700,7.328350,351,106.621755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
409,World,M,79.04,A,Wasana Trade Point,"Wasana Trade Point, Temple Road, Negombo, LKA",Wasana Trade Point,POI,Business Facility,Wasana Trade Point,...,ENG,LK1,LK12,LK1224,None,POINT (79.85915 7.19703),79.859150,7.197030,61483,24.739219
410,World,T,77.68,A,Post Office-Wadduwa,"Post Office-Wadduwa, Wadduwa Station Road, Wad...",Post Office-Wadduwa,POI,Post Office,Post Office-Wadduwa,...,ENG,LK1,LK13,LK1321,None,POINT (79.92945 6.66347),79.929450,6.663470,288835,58.271528
411,World,M,72.17,A,Madusanka,"Madusanka, A8, Wekada West, LKA",Madusanka,POI,Grocery,Madusanka,...,ENG,LK1,LK13,LK1321,None,POINT (79.91315 6.71011),79.913150,6.710110,238173,12.981828
412,World,T,73.63,A,Post Office-Beruwala,"Post Office-Beruwala, Mangala Road, Paranakade...",Post Office-Beruwala,POI,Post Office,Post Office-Beruwala,...,ENG,LK1,LK13,LK1321,None,POINT (79.98347 6.47290),79.983470,6.472900,274287,13.661556


In [84]:
tourism_ls = list(set(tourism.NN))

# prints NN list for tourism
tourism_ls

[42497,
 300036,
 31748,
 288772,
 305671,
 181256,
 217095,
 76810,
 152068,
 104460,
 90128,
 153105,
 263184,
 108563,
 177172,
 187925,
 162836,
 257047,
 193559,
 193561,
 288281,
 136219,
 224283,
 214550,
 196638,
 83490,
 16420,
 113188,
 61483,
 292396,
 169005,
 207918,
 209452,
 36909,
 123439,
 288310,
 117816,
 282681,
 51768,
 7228,
 62,
 54334,
 239678,
 112193,
 80962,
 288835,
 193607,
 302152,
 69193,
 38986,
 131145,
 184911,
 138319,
 303183,
 23634,
 13394,
 254036,
 181330,
 224855,
 222296,
 111705,
 246873,
 234583,
 99931,
 144989,
 63064,
 311896,
 238173,
 116834,
 287844,
 14950,
 220264,
 125033,
 239721,
 208490,
 95344,
 90737,
 15474,
 207990,
 108664,
 67195,
 94336,
 288018,
 23684,
 238215,
 69768,
 281227,
 33420,
 303757,
 141966,
 277648,
 44179,
 114836,
 162965,
 264854,
 139927,
 203411,
 26265,
 44703,
 55968,
 313503,
 92834,
 78499,
 182436,
 129700,
 51362,
 42151,
 1704,
 297633,
 153770,
 122540,
 89263,
 280754,
 250547,
 324788,
 171701,

In [88]:
# calculate OD from cities to tourism destinations
# these routes will be called later to compare with travel time with traffic.
# 'CT' stands for City-to-Tourism
OD_CT = gn.calculate_OD(G_time,
                     city_ls, 
                     tourism_ls, 
                     fail_value = 9999999)

OD_CT

array([[25364.37009561, 18737.34638754, 15975.12334353, ...,
        28810.14522054, 26663.83490572, 21559.01609162],
       [ 4874.49300288,  6499.40316324,  8710.25593521, ...,
         8863.97414021, 16166.84028417,  1690.42035062],
       [16629.72602416, 10002.70231609,  7240.47927208, ...,
        22279.3883181 , 17929.19083427, 15105.8345285 ],
       ...,
       [21677.58784109, 15050.56413302, 12288.341089  , ...,
        27327.25013502, 22020.74622253, 20153.69634543],
       [18291.19098218, 11753.62187269,  9363.02932361, ...,
        22840.45464992,  5478.03400394, 17742.67901863],
       [ 7456.91141231,  1087.00268588,  5712.53811911, ...,
        13888.00517622, 10748.93327698,  6714.45138662]])

In [89]:
OD_CT.shape

(26, 336)

In [90]:
# convert the OD Matrix numpy array into a pandas dataframe using minutes as the measure
# output: minutes from airport (18713) to each destination NN id (n=336)
OD_CT = OD_CT / 60

OD_dfCT = pd.DataFrame(OD_CT, columns = tourism_ls, index = city_ls)

OD_dfCT

,42497,300036,31748,288772,305671,181256,217095,76810,152068,104460,...,218611,29684,162293,205814,285175,112630,229369,314357,152574,171007
59905,422.739502,312.289106,266.252056,373.418703,252.142350,203.868217,197.206933,327.895657,367.376943,417.738580,...,395.508548,194.565262,300.378680,454.666744,326.768450,201.160627,283.209337,480.169087,444.397248,359.316935
237826,81.241550,108.323386,145.170932,139.964649,233.317890,181.056883,178.586724,27.240227,34.940759,202.230905,...,287.395505,181.285393,202.875500,282.807152,91.804093,177.688620,72.271534,147.732902,269.447338,28.173673
164101,277.162100,166.711705,120.674655,227.841301,106.559600,58.290816,51.629532,227.353332,258.530995,272.161179,...,249.925799,48.987861,154.801279,309.089343,181.191048,55.583226,182.667011,371.323139,298.819847,251.763909
262918,168.271917,291.405900,328.253446,277.628390,416.400404,364.139397,361.669238,194.203501,154.719175,203.808562,...,334.392878,364.367907,350.645503,251.167746,274.886607,360.771133,239.231767,41.927031,264.527559,166.193712
19847,145.829120,131.625858,182.571712,86.942360,249.257270,236.163521,229.168931,223.562550,208.236599,0.831814,...,172.580879,222.341505,147.945130,121.407335,113.254995,233.455931,226.907201,244.024096,123.695439,211.199665
281879,157.287057,247.423120,284.270666,279.064383,372.417624,320.156617,317.686458,150.220721,110.736395,246.927320,...,377.851615,320.385127,341.975234,294.626483,230.903827,316.788354,195.248988,2.071410,307.986296,122.210932
210210,20.346878,149.603236,199.369043,181.244499,287.516001,237.670860,235.200701,112.939720,82.754357,126.322075,...,297.778413,235.483504,257.073611,214.553281,133.083944,234.302597,157.811333,152.449439,227.913094,88.640163
48165,145.493215,257.310983,312.895249,213.230302,355.832153,362.817196,359.492467,238.086056,207.900694,131.431726,...,257.320963,352.665041,278.268667,174.095831,240.791690,359.448933,282.957669,125.842934,187.455644,213.786499
215717,280.635047,170.184652,107.148960,231.314248,17.643579,111.438347,104.443757,239.638365,262.109539,255.830147,...,161.009777,96.640731,119.931994,233.026833,184.663995,108.730757,235.455754,374.901683,226.035905,255.342453
106667,107.923951,125.305705,151.818688,161.847069,239.965646,178.169327,176.517275,2.293495,41.762031,226.586139,...,309.277925,187.933149,209.523256,304.689572,113.686513,174.801064,45.776182,154.554175,291.329758,33.702023


### part 4: collect traversed paths
#### example from gn iceland tutorial - https://github.com/worldbank/GOSTnets/blob/master/Tutorials/EXAMPLE%20Finding_links_between_pairs.ipynb

#### 4.1: get routes from airports to cities

#### 4.2: get routes from cities to cities

#### 4.3: get routes from cities to tourism destinations

In [ ]:
# re-read in G_time
G_time = nx.read_gpickle(os.path.join(pth, r'G_time.pickle'))
# make sure origins and destinations have been snapped to network and created as list objects.

In [104]:
# create separate object with G_time nodes and edges:
nodes_gdf = gn.node_gdf_from_graph(G_time)
edges_gdf = gn.edge_gdf_from_graph(G_time)

In [107]:
# example of one OD pair
# create object of all nodes in shortest path
obj_nodes = nx.shortest_path(G_time, 
                             source=city_ls[0], 
                             target=tourism_ls[0], 
                             weight='time')
print(city_ls[0])
print(tourism_ls[0])
# this is a list of the network graph nodes that, connected, make up the shortest path from the first origin and first destination
print(obj_nodes)


59905
42497
[59905, 178723, 257367, 119234, 91554, 149298, 176520, 229307, 49500, 184671, 164349, 258571, 136969, 178905, 180099, 212250, 210359, 257737, 210191, 176519, 313423, 194153, 240362, 100152, 111419, 296965, 231717, 221662, 110592, 176187, 21871, 205395, 259259, 78204, 309080, 326997, 88357, 213846, 264207, 79091, 2102, 231208, 92176, 154091, 294690, 140709, 227299, 28664, 189276, 161030, 184116, 110907, 286116, 323478, 213579, 230494, 39361, 137697, 207094, 290389, 166143, 255778, 156605, 321771, 261197, 72694, 226265, 38017, 197436, 129117, 124574, 303760, 230931, 70100, 310539, 155131, 324667, 152607, 81373, 113929, 165099, 169677, 270659, 63712, 96967, 59998, 157702, 80538, 171738, 210444, 125557, 61345, 4420, 119668, 243206, 245100, 189192, 259933, 226346, 326718, 171996, 75519, 52506, 271159, 151212, 64717, 26263, 245260, 201232, 250851, 91826, 159538, 45952, 311600, 50583, 256254, 301869, 63548, 198663, 260424, 6361, 166325, 223984, 147881, 44414, 310185, 72366, 124195

when running the following code block, the script stops at the first OD pair

In [110]:
# calculate line strings connecting all origins to all destinations
all_res=[]
all_connections=[]
oIdx = 0 #city ID count

for c in city_ls:
    oIdx = oIdx + 1
    print(f'{oIdx} of {len(city_ls)}')
    for t in tourism_ls:
        obj_nodes = nx.shortest_path(G_time, source=c, target=t, weight="time")
        all_edges = []
        for idx in range(0, len(obj_nodes) - 1):
            start_node = obj_nodes[idx]
            end_node = obj_nodes[idx + 1]
            cur_edge = edges_gdf.loc[(edges_gdf['stnode'] == start_node) & (edges_gdf['endnode'] == end_node), 'geometry'].iloc[0]
            all_edges.append(cur_edge)
            all_connections.append([start_node, end_node, cur_edge])
        all_res.append([c, t, MultiLineString(all_edges)])

1 of 26


KeyboardInterrupt: 

In [ ]:
# write all routes to csv file
all_results = pd.DataFrame(all_res, columns['O', 'D', 'geometry'])
all_results.to_csv(os.path.join(pth, 'lka_data', 'OD_links_city2tourism.csv'))

In [ ]:
# tabulate use of individual links and write to csv file
all_conn = pd.DataFrame(all_connections, columns=['start','node','geometry'])
all_connections_count = pd.DataFrame(all_conn.groupby(['start','node']).count())
all_connections_count.reset_index(inplace=True)
all_connections_first = pd.DataFrame(all_conn.groupby(['start','node']).first())
all_connections_first.reset_index(inplace=True)
all_connections_first['count'] = all_connections_count['geometry']

In [ ]:
all_connections_first.to_csv(os.path.join(pth, "tutorial_data","OD_links_usage.csv"))
